In [ ]:
import glob
import os

import contextily as ctx
import geopandas as gpd
import imageio
import matplotlib.pyplot as plt
import matplotlib.colors
import pandas as pd
import pygifsicle

In [ ]:
flows = pd.read_csv(os.path.join('data','link_use_central_oxford.csv')) \
    .rename(columns={'edgeID': 'link', 'vehicles': 'flow'})
flows.head()

In [ ]:
unique_link_ids = flows.link.unique()
unique_link_ids

In [ ]:
len(unique_link_ids)

In [ ]:
links = gpd.read_file(os.path.join('data','fullNetworkWithEdgeIDs.shp')) \
    .drop(columns=['Anode', 'Bnode', 'CP', 'iDir', 'SRefE', 'SRefN', 'Distance', 'FFspeed', 'FFtime', 'IsFerry']) \
    .rename(columns={'EdgeID': 'link'})
links.head()

In [ ]:
hourly_maxes = flows.drop(columns=['link']).groupby('hour').max()
hourly_maxes.head()

In [ ]:
flow_max = flows.flow.max()
flow_max

In [ ]:
def plot_map(interval, links, flow_max, output_filename):
    ax = links.plot(
        figsize=(10, 12),
        column='flow', 
        cmap='YlOrRd',    
        norm=matplotlib.colors.Normalize(vmin=0.0001, vmax=flow_max),
        legend=True,
        legend_kwds={'label': "Flow volume", 'orientation': "horizontal"}
    )

    plt.title('{}:00'.format(interval), fontsize=16)
    ctx.add_basemap(ax, crs=links.crs);
    
    # Save to file
    plt.savefig(output_filename)
    plt.close()

In [ ]:
for interval_key, interval_name in [
    ('MIDNIGHT', '00'),
    ('ONEAM', '01'),
    ('TWOAM', '02'),
    ('THREEAM', '03'),
    ('FOURAM', '04'),
    ('FIVEAM', '05'),
    ('SIXAM', '06'),
    ('SEVENAM', '07'),
    ('EIGHTAM', '08'),
    ('NINEAM', '09'),
    ('TENAM', '10'),
    ('ELEVENAM', '11'),
    ('NOON', '12'),
    ('ONEPM', '13'),
    ('TWOPM', '14'),
    ('THREEPM', '15'),
    ('FOURPM', '16'),
    ('FIVEPM', '17'),
    ('SIXPM', '18'),
    ('SEVENPM', '19'),
    ('EIGHTPM', '20'),
    ('NINEPM', '21'),
    ('TENPM', '22'),
    ('ELEVENPM', '23')
    ]:
    hour_flows = flows[flows.hour == interval_key].copy()
    hour_flows = links.merge(hour_flows, on='link')

    plot_map(interval_name, hour_flows, flow_max, os.path.join('vis', interval_name))
    print("Done", interval_name)

In [ ]:
gif_path = 'movie.gif'
filenames = glob.glob(os.path.join('vis','*.png'))

images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave(os.path.join(gif_path), images)

In [ ]:
pygifsicle.optimize(gif_path)